In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Prerequisite

In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Reshape
from tensorflow.keras import backend as K

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Load the train and test data splits

In [ ]:
GER_test = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_train.mat')['GER_train'])).transpose()
test_frame = pd.DataFrame(GER_test).sort_values(1)
train_frame = pd.DataFrame(GER_train).sort_values(1)

In [ ]:
print(f"x_train shape: {GER_train.shape} - y_train shape: {GER_train.shape}")
print(f"x_test shape: {GER_test.shape} - y_test shape: {GER_test.shape}")

x_train shape: (6981, 21) - y_train shape: (6981, 21)
x_test shape: (6248, 21) - y_test shape: (6248, 21)


In [ ]:
def get_data_labels(split):
    x = pd.DataFrame(split)
    labels = x[0].values.astype(np.uint8)
    del x[0],x[1]
    data = x.values 
    return data, labels
    
x_train, y_train = get_data_labels(train_frame.values.tolist())
x_test, y_test = get_data_labels(test_frame.values.tolist())

In [ ]:
df = pd.DataFrame(x_train)
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
df2 = pd.DataFrame(x_test)
scaler = StandardScaler()
x_test = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns)

In [ ]:
print(f"Size of the training set: {len(x_train)}")

Size of the training set: 6981


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(f"Size of the x training set: {len(x_train)}")
print(f"Size of the x validation set: {len(x_val)}")
print(f"Size of the y training set: {len(y_train)}")
print(f"Size of the y validation set: {len(y_val)}")

Size of the x training set: 5584
Size of the x validation set: 1397
Size of the y training set: 5584
Size of the y validation set: 1397


In [ ]:
print(x_train[:5])
print(y_train[:5])

            0         1         2         3         4         5         6   \
6565 -1.076680 -1.065748 -1.012843 -0.799906 -0.713923 -0.710367 -1.932554   
4003  0.582947  1.135969  1.297658  1.015694  0.868769 -0.130759 -1.806106   
1010 -0.545586 -0.653364 -0.430347 -0.782925 -0.775639 -0.323475 -0.652264   
4617 -0.566326 -0.578170 -0.686170 -0.379677 -0.210357 -1.287966  0.392749   
931   0.245015  0.061639  0.443537 -0.616326 -0.935091  0.915754  0.721268   

            7         8         9         10        11        12        13  \
6565  2.396384 -0.985383 -0.853229 -0.838086 -0.790418 -0.770441  1.082992   
4003  1.601756 -0.026166  1.974222  2.026149  2.399590  2.084844 -0.682288   
1010  0.034567 -1.103360  0.015721  0.285905  0.997402  0.964844 -2.012156   
4617 -0.268374 -0.030353 -0.463698 -0.261271  1.170901  1.321986 -2.641625   
931  -0.828500 -0.803408 -0.062883 -0.065124 -0.179201 -0.313890  0.995570   

            14        15        16        17        18  
6565 

Early stopping to prevent over-fitting

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)





# Function Calls

In [ ]:
input_shape = (19,)

In [ ]:
# Define the sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], 19), mean=0., stddev=0.004)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

# def sampling(args):
#     """Reparameterization trick. Instead of sampling from Q(z|X), 
#     sample eps = N(0,I) z = z_mean + sqrt(var)*eps.

#     Parameters:
#     -----------
#     args: list of Tensors
#         Mean and log of variance of Q(z|X)

#     Returns
#     -------
#     z: Tensor
#         Sampled latent vector
#     """

#     z_mean, z_log_var = args
#     eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1.0, name='epsilon')
#     z = z_mean + tf.exp(z_log_var / 2) * eps
#     return z

# Encoder

In [ ]:
inputs = keras.layers.Input(shape=input_shape, name='input')
x = keras.layers.Dense(64, activation='relu')(inputs)
z_mean = keras.layers.Dense(19, name='z_mean')(x)
z_log_var = keras.layers.Dense(19, name='z_log_var')(x)

In [ ]:
z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
encoder = keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense_57 (Dense)               (None, 64)           1280        ['input[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 19)           1235        ['dense_57[0][0]']               
                                                                                                  
 z_log_var (Dense)              (None, 19)           1235        ['dense_57[0][0]']               
                                                                                            

# Decoder

In [ ]:
#Giving input to decoder and initialising shape
latent_inputs = keras.layers.Input(shape=(19), name='z_sampling')
middle=keras.layers.Dense(64, activation='relu')(latent_inputs)
x = keras.layers.Dense(19, activation='relu')(middle)

# instantiate decoder model
decoder = keras.Model(latent_inputs, x, name='decoder')

# Implementing VAE without Loss

In [ ]:
outputs = decoder(encoder(inputs)[2])  # Select the Z value from outputs of the encoder
vae = keras.Model(inputs, outputs, name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 19)]              0         
                                                                 
 encoder (Functional)        [(None, 19),              3750      
                              (None, 19),                        
                              (None, 19)]                        
                                                                 
 decoder (Functional)        (None, 19)                2515      
                                                                 
Total params: 6,265
Trainable params: 6,265
Non-trainable params: 0
_________________________________________________________________


# Initialising Loss 

In [ ]:
autoencoder = Model(inputs, outputs)

In [ ]:
reconstruction_loss = K.mean(K.square(outputs - inputs), axis=-1)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)


# Training VAE

In [ ]:
autoencoder.add_loss(vae_loss)
autoencoder.summary()
autoencoder.compile(optimizer='adam')

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 encoder (Functional)           [(None, 19),         3750        ['input[0][0]']                  
                                 (None, 19),                                                      
                                 (None, 19)]                                                      
                                                                                                  
 decoder (Functional)           (None, 19)           2515        ['encoder[0][2]']                
                                                                                           

In [ ]:
autoencoder.fit(x_train, epochs=50, batch_size=32, validation_data=(x_val, ),callbacks=[es])

Epoch 1/50
175/175 [==============================] - 2s 3ms/step - loss: 1.8737 - val_loss: 1.1360
Epoch 2/50
175/175 [==============================] - 0s 3ms/step - loss: 0.9722 - val_loss: 0.8473
Epoch 3/50
175/175 [==============================] - 0s 2ms/step - loss: 0.7561 - val_loss: 0.6680
Epoch 4/50
175/175 [==============================] - 0s 2ms/step - loss: 0.6321 - val_loss: 0.5943
Epoch 5/50
175/175 [==============================] - 0s 2ms/step - loss: 0.5746 - val_loss: 0.5495
Epoch 6/50
175/175 [==============================] - 0s 2ms/step - loss: 0.5404 - val_loss: 0.5223
Epoch 7/50
175/175 [==============================] - 1s 4ms/step - loss: 0.5185 - val_loss: 0.5070
Epoch 8/50
175/175 [==============================] - 1s 5ms/step - loss: 0.5028 - val_loss: 0.4904
Epoch 9/50
175/175 [==============================] - 1s 5ms/step - loss: 0.4892 - val_loss: 0.4824
Epoch 10/50
175/175 [==============================] - 1s 4ms/step - loss: 0.4816 - val_loss: 0.4732

# Extracting the Encoder part of VAE and freezing them

In [ ]:
encoder = Model(inputs, z)
encoder.trainable = False
encoder.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense_57 (Dense)               (None, 64)           1280        ['input[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 19)           1235        ['dense_57[0][0]']               
                                                                                                  
 z_log_var (Dense)              (None, 19)           1235        ['dense_57[0][0]']               
                                                                                           

In [ ]:
classifier_input = Input(shape=(19,))
encoded_input = encoder(classifier_input)
x1=Dense(128,activation='relu')(encoded_input)
x2=Dense(64,activation='relu')(x1)
x3=Dense(32,activation='relu')(x2)
x4=Dense(16,activation='relu')(x3)
x5=Dense(8,activation='relu')(x4)
x6=Dense(4,activation='relu')(x5)
output_layer = Dense(1, activation='sigmoid')(x6)

classifier = Model(classifier_input, output_layer)
classifier.summary()
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val),callbacks=[es])


Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 19)]              0         
                                                                 
 model_14 (Functional)       (None, 19)                3750      
                                                                 
 dense_60 (Dense)            (None, 128)               2560      
                                                                 
 dense_61 (Dense)            (None, 64)                8256      
                                                                 
 dense_62 (Dense)            (None, 32)                2080      
                                                                 
 dense_63 (Dense)            (None, 16)                528       
                                                                 
 dense_64 (Dense)            (None, 8)                 136

# Measuring Accuracy

In [ ]:
accuracy = classifier.evaluate(x_train, y_train)[1]
print(f"Train accuracy: {round(accuracy * 100, 2)}%")

175/175 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8598
Train accuracy: 85.98%


In [ ]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

196/196 [==============================] - 0s 2ms/step - loss: 0.4201 - accuracy: 0.8211
Test accuracy: 82.11%
